# Operator overloading

Now that you remembered how to implement classes and declare methods, let's look at how to modify them. This is not limited to custom operators but Python operators can be modified as well

## \__init__

The init operator is probably the most common to be overloaded. This class is called by default when calling a class to build a new object, even if it's not specifically implemented. Remember that init always takes self as the first argument.

For this example, let's consider a vector (a tuple of N coordinates). The length, or number of coordinates determines the dimension of the vector. 


In [1]:
class Vector:
    def __init__(self,*args):
        self.coords = args
v = Vector(0,0)
print(v.coords)

v.coords = (1,1)
print(v.coords)

v.__init__(5,5,5,5,5)
print(v.coords)

(0, 0)
(1, 1)
(5, 5, 5, 5, 5)


Note that self.coords = ... establishes a new attribute name in the namespace
of the constructed object and initializes it to the tuple args. 
        
Like the methods that we will discuss below, we don't call \__init__ directly,
but Python does, automatically, when we "call the Vector class" to construct a
new object from that class.

We can also use \__init__ another way in Python: we can call \__init__
explicitly on any existing Vector object, to reinitialize its coords attribute.

## \__len__

We can call the len function on any object. It is automatically imported from the builtins module and most objects (including custom ones) will have it. (note: Int is one of the object types that won't return a len value but instead raise a TypeError)

No let's use this knowledge to have Vector overload the len method.

In [2]:
class Vector:
    def __init__(self,*args):
        self.coords = args

    def __len__(self):
        print('Calling __len__')
        return len(self.coords)


v = Vector(0,0)
print(len(v))

Calling __len__
2


## __bool__

Whenever Python needs to interpret some object as a boolean value, it calls the
parameterless method \__bool__ on the object. For example, if we write an "if"
or "while" statement whose boolean test is just an object, Python attempts to
call the \__bool__function on that object to determine whether its boolean
equivalent represents True or False.

Using the definition of Vector above (defining just __init__ and __len__) and
the test function below


In [3]:
def test(x):
    print('x\'s boolean equivalent is ')
    if x:
        print(True)
    else:
        print(False)


v = Vector(0,0)
test(v)

v = Vector(2,2)
test(v)

v = Vector()
test(v)


x's boolean equivalent is 
Calling __len__
True
x's boolean equivalent is 
Calling __len__
True
x's boolean equivalent is 
Calling __len__
False


The actual rule in Python for evaluating an object as if it were a boolean
value is to first try to call \__bool__ on the object (which cannot be done
above, because a \__bool__ method is not defined) and return its result; if that
method is not present, Python instead returns whether the object's len is != 0.
If there is no \__len__ function Python just returns True (the mechanism to do
this is actually related to inheritance, a topic we will cover later in the
quarter).

This rule explains why we can test str, list, tuple, set, and dict objects as
booleans: all define \__len__, and if any is empty (len = 0) it represents
False; if any is not empty, it represents True.  Also, the object None has a
boolean value of False (the NoneType class specifies a \__bool__ method that
always return False).

So, when v = Vector(0,0) or v = Vector(2,2), len(v) is 2 which is != 0, so it
is treated as True. When v = Vector(), len(v) is 0 which is not != 0 so it is
treated as False.

But suppose that we want to interpret vectors as boolean values differently. We
will define Vector so that an instance is True if self.coords is not the
origin, regardless of the vector's dimension: so, it is True if any of the
coordinates in self.coords is not 0.

In [10]:
class Vector:
    def __init__(self,*args):
        self.coords = args

    def __len__(self):
        print('Calling __len__')
        return len(self.coords)

    def __bool__(self):
        print('Calling __bool__')
        return any( v!=0 for v in self.coords )
    
v = Vector(0,0)
test(v)

v = Vector(2,2)
test(v)

v = Vector()
test(v)

x's boolean equivalent is 
Calling __bool__
False
x's boolean equivalent is 
Calling __bool__
True
x's boolean equivalent is 
Calling __bool__
False


However, since the class vector is supposed to represent a mathematical vector (quantity that represents a point in N-dimensional space) we want it to be false whenever the coordinates equals to zero. Therefore we can modify the len operator to instead return the distance from the origin to the tip of the vector

In [13]:
from math import *
class Vector:
    def __init__(self,*args):
        self.coords = args

    def __len__(self):
        return int(math.sqrt( sum( v**2 for v in self.coords ) ))


    
v = Vector(0,0)
test(v)

v = Vector(2,2)
test(v)

v = Vector(0,0,0)
test(v)

x's boolean equivalent is 
False
x's boolean equivalent is 
True
x's boolean equivalent is 
False


Now we don't need a bool operator since len already takes care of it by itself.

##  \__repr__ and \__str__

Python can call two methods that should return string representations of an
object. The __str__ method is called when the conversion function str is called:
so again, this is like len: Python translates str(x) into type(x).\__str__(x).

For example, the print and format functions automatically calls str on all their
arguments. If there is no \__str__ method in the argument's class, Python
tries calling the \__repr__ method as a backup to produce the string. If we call
repr(x) Python returns x.\__repr__() for this method, similarly to what it does
for len(x) and str(x)). If there is no \__repr__method, Python reverts to its
standard method for computing a string value objects: a bracket string with the
name of the object's class and the location of this object in memory (as a
hexidecimal number). 

In [14]:
v = Vector(0,0)
print(v)


The convention for \__repr__ is that it returns a string, which if passed as the
argument to the eval function, would produce an object with the same state. So
if v = Vector(0,0) the repr(v) should return 'Vector(0,0)'. For Vector, we
can define \__repr__ as follows

In [15]:
class Vector:
    def __init__(self,*args):
        self.coords = args

    def __len__(self):
        print('Calling __len__')
        return len(self.coords)

    def __bool__(self):
        print('Calling __bool__')
        return any( v!=0 for v in self.coords )
    def __repr__(self):
        return 'Vector({})'.format(','.join(str(c) for c in self.coords))
v = Vector(0,0)
print(v)

Vector(0,0)


In [16]:
x = eval(repr(v))
print(type(x),x)

<class '__main__.Vector'> Vector(0,0)


Here x refers to a different object than v, whose
state is the same as v's state: x is v evaluates to False.

Let's use this knowledge and overload str.

In [28]:
class Vector:
    def __init__(self,*args):
        self.coords = args

    def __len__(self):
        return len(self.coords)

    def __bool__(self):
        return all( v==0 for v in self.coords )

    def __repr__(self):
        return 'Vector('+','.join(str(c) for c in self.coords)+')'

    def __str__(self):
        return '('+str(len(self))+')'+str(list(self.coords))      # using +
       #return '({d}){c}'.format(d=len(self),c=list(self.coords)) # using format

        
        
v = Vector(0,0)
print(str(v))

(2)[0, 0]


## Relational operators: 
__lt__ (__gt__, __le__, __ge__, __eq__, __ne__): < (>, <=, >=, ==, and !=)

Python translates any occurrence of a relational operator into a call on the appropriate method for its LEFT operand:
x < y is translated to x.\__lt__(y) which by the Fundamental Equation of Object
Oriented Programming (FEOOP) is translated into type(x).\__lt__(x,y) or,
assuming Vector is a class and x is of type Vector, Vector.\__\lt__(x,y).

So, the type of first/left operand of < determines which class calls its \__lt__
method.

When studying relational operators, we will first look at comparing objects from
the same class, and then we will extend our understanding to how Python compares
objects from different classes (which is a bit more subtle).

In [29]:
x = Vector(0,0)
y = Vector(2,2)
print(x < y)

TypeError: '<' not supported between instances of 'Vector' and 'Vector'

This exception is raised because vector is unorderable. That means that when calling upon a compare operator, there is none to be found. Besides, what would it compare, the lenght? the first coordinate?, first we need to create an "orderable"operator, that is something that you can order by, and then instruct the compare operator to use that as referrence.

In [30]:
class Vector:
    def __init__(self,*args):
        self.coords = args

    def __len__(self):
        return len(self.coords)

    def __bool__(self):
        return all( v==0 for v in self.coords )

    def __repr__(self):
        return 'Vector('+','.join(str(c) for c in self.coords)+')'

    def __str__(self):
        return '('+str(len(self))+')'+str(list(self.coords))      # using +
       #return '({d}){c}'.format(d=len(self),c=list(self.coords)) # using format
    def distance(self):
        return math.sqrt( sum( v**2 for v in self.coords ) )

    def __lt__(self,right):
        return self.distance() < right.distance()
x = Vector(0,0)
y = Vector(2,2)
print(x < y)

True


Notice the arguments for lt, it needs self and right. I named the second parameter here right because it is the value on the right-
hand side of the < operator; it can be named anything (e.g., qq17), so long as
that same name is used for the second parameter in the body of the function.

Notice the < operator in \__lt__ is NOT RECURSIVE! Python calls the \__lt__
method above when comparing two objects constructed from the Vector class; but
inside this method the < operator is called on two float value returned by the
sqrt function in distance, so it calls the \__lt__ method defined in the float
class:

   self.distance() < right.distance()
   
   
Now let's overload all other comparison operators

In [32]:
class Vector:
    def __init__(self,*args):
        self.coords = args

    def __len__(self):
        return len(self.coords)

    def __bool__(self):
        return all( v==0 for v in self.coords )

    def __repr__(self):
        return 'Vector('+','.join(str(c) for c in self.coords)+')'

    def __str__(self):
        return '('+str(len(self))+')'+str(list(self.coords))      # using +
       #return '({d}){c}'.format(d=len(self),c=list(self.coords)) # using format

    def distance(self):
        return math.sqrt( sum( v**2 for v in self.coords ) )

    def __lt__(self,right):
        if type(right) is Vector:
            return self.distance() < right.distance()
        elif type(right) in (int,float):
            return self.distance() < right
        else:
            return NotImplemented

    def __gt__(self,right):
        if type(right) is Vector:
            return self.distance() > right.distance()
        elif type(right) in (int,float):
            return self.distance() > right
        else:
            return NotImplemented

    def __eq__(self,right):
        return self.coords == right.coords
    
    def __le__(self,right):
        return self < right or self == right

    def __ge__(self,right):
        return self > right or self == right

    def __neg__(self):
        return Vector( *(-c for c in self.coords) )
    
x = Vector(0,0)
y = Vector(2,2)
print(x < y)
print(x == y)
print(x != y)
print(x > y)
print(x <= y)
print(x >= y)
print(not(x))

True
False
True
False
True
False
False


## Arithmetic operators

By now you should understand that every operator has it's declaration inside python. Arithmetic operators are not excluded. For example calling x + y causes x.\__add__(y) to be returned. First let's see how our whole vector class looks like with arithmetic operators and then we'll explain one by one:

In [35]:
import math
#from goody import type_as_str

class Vector:
    def __init__(self,*args):
        self.coords = args

    def __len__(self):
        return len(self.coords)

    def __bool__(self):
        return all( v==0 for v in self.coords )

    def __repr__(self):
        return 'Vector('+','.join(str(c) for c in self.coords)+')'

    def __str__(self):
        return '('+str(len(self))+')'+str(list(self.coords))      # using +
       #return '({d}){c}'.format(d=len(self),c=list(self.coords)) # using format

    def distance(self):
        return math.sqrt( sum( v**2 for v in self.coords ) )

    def __lt__(self,right):
        if type(right) is Vector:
            return self.distance() < right.distance()
        elif type(right) in (int,float):
            return self.distance() < right
        else:
            return NotImplemented

    def __gt__(self,right):
        if type(right) is Vector:
            return self.distance() > right.distance()
        elif type(right) in (int,float):
            return self.distance() > right
        else:
            return NotImplemented

    def __eq__(self,right):
        return self.coords == right.coords
    
    def __le__(self,right):
        return self < right or self == right

    def __ge__(self,right):
        return self > right or self == right

    def __neg__(self):
        return Vector( *(-c for c in self.coords) )

    def __pos__(self):
        return self

    def __abs__(self):
        return Vector( *(abs(c) for c in self.coords) )
    
    def __add__(self,right):
        if type(right) not in (Vector,int,float):
            return NotImplemented
        if type(right) in (int,float):
            return Vector( *(c+right for c in self.coords) )
        else:
            assert len(self) == len(right), 'Vector.__add__: operand self('+str(self)+') has different dimension that operand right('+str(right)+')'
            return Vector( *(c1+c2 for c1,c2 in zip(self.coords,right.coords)) )
    
    def __radd__(self,left):
        if type(left) not in (int,float): # see note below
            return NotImplemented
        return Vector( *(left+c for c in self.coords) )
    
    def __iadd__(self,right):
        if type(right) not in (Vector,int,float):
             return NotImplemented
        if type(right) in (int,float):
             return Vector( *(c+right for c in self.coords))
        else:
             assert len(self) == len(right), 'Vector.__add__: operand self('+str(self)+') has different dimension that operand right('+str(right)+')'
             return Vector( *(c1+c2 for c1,c2 in zip(self.coords,right.coords)))


### Unary operators

When Python recognizes a unary arithmetic operator (or a binary arithmetic operator, see the next section) it translates it into the appropriate method call for the class/type of its argument: for example, it translates -x into x.\__neg__() and then into type(x).\__neg__(x)
and if x is a Vector, finally into Vector.\__neg__(x).

Suppose that we wanted the \__neg__ operator for Vector to return a Vector
object with all of its coords negated. Generally, as described above,__neg__
SHOULD NOT MUTATE ITS OPERAND but should leave its operand unchanged and return
a new Vector object, whose state is initialized to the appropriate one. Here is
a \__neg__ method with exactly these semantics. Note it returns a newly
constructed  object (of class Vector) with the appropriate contents,

    def __neg__(self):
        return Vector( *(-c for c in self.coords) )

Notice the use of tuple comprehension to create the appropriate negated tuple,
and the use of * to translate the n-tuple comprehension into n different
positional arguments needed in the call to Vector's \__init__. That is, calling
f(1,*(2,3,4),5) is equivalent to calling f(1,2,3,4,5). Here is an example of it
code running

In [36]:
v = Vector(1,-2,3)
print(v)
print(-v)
print(v)
## note that the object v refers to was not mutated by __neg__


(3)[1, -2, 3]
(3)[-1, 2, -3]
(3)[1, -2, 3]


There are two other unary arithmetic operators that we can overload: + and ~
whose methods go by the names \__pos_ and \__invert__. For Vectors, pos should
return the same vector. Invert is a bit-wise operator, which we won't overload.
So writing ~v would raise an exception.

In [39]:
print(+v)
print(~v)

(3)[1, -2, 3]


TypeError: bad operand type for unary ~: 'Vector'

In addition, while discussing arithmetic here, the following unary functions 
(all defined in the math module) abs, round, floor, ceil, trunc work by calling
one of these special methods on its argument (much like len, described above):
so abs(x) returns as its result x.\__abs__(). Therefore, if we defined the
following \__abs__ method in the Vector class, it returns an object constructed
from the Vector class (containing the absolute values of all the coordinates)

In [40]:
v = Vector(1,-2,3)
print(v)
print(abs(v))
print(v)

(3)[1, -2, 3]
(3)[1, 2, 3]
(3)[1, -2, 3]


### Binary operators

We continue through our Vector class, and there are still operators that are being overloaded. 

Binary arithmetic operators, like relational operators, are written in between
their two operands. Python translates the call x + y into x.\__add__(y) and then
by FEOOP into type(x).\__add__(x,y). As with the relational operators and unary
arithmetic operators, neither operand should be mutated, and the method should
return a new object initialized with the correct state. Here is an example of
the + operator overloaded for Vector: to work correctly, the right operand must
also be of type Vector and both must have the same number of coordinates (len
of the coords attribute) and the resulting Vector object has that common length,
with coordinates that are the pairwise sum of the coordinates in the two
Vectors. 

In [44]:
v1 = Vector(0,1)
v2 = Vector(2,2)
print(v1+v2)
print(v1+1)

(2)[2, 3]
(2)[1, 2]


For every binary arithmetic operator, Python also allows us to define a "right"
(sometimes known as "reversed") version of it: where the method name is prefixed
by an r: so \__add__ has an related \__radd__ method ("right/reverse add"). Here
is how we could define \__radd__ in the Vector class to successfully compute
expressions of the form int() + Vector().

    def __radd__(self,left):
        if type(left) not in (int,float):               # see note (1) below
            return NotImplemented
        return Vector( *(left+c for c in self.coords) ) # see note (2) below

When Python evaluates 1+v, it translates it into 1.\__add__(v) and then by FEEOP
into tries int.\__add(1,v); it doing so returns NotImplmemented or raises an
exception because the int class doesn't know how to operate on Vector operands.
Then, Python translates the + into v.\__radd_(1) and into type(v).\__radd__(v,1)
using  "right/reversed-operand dispatch". This methods determines what to do if
the left operand is an int/float. In the method below, relating to the + 
operator, the self parameter is the right operand and the left parameter is the
left operand. We could rewrite the headers as def \__add__(left,right) and
def \__radd__(right,left) and replacing self appropriately in the method bodies.

In [45]:
v = Vector(0,0)
print(1+v)

(2)[1, 1]


### Incrementing arithmetic delimiters

There is still one last operator to look at: iadd which represents +=.

Recall that the meaning of x += y is similar to x = x + (y). We parenthesize y
in case it is an expression that contains any operators whose precedence is
lower than +. When Python executes x += y, it tries to execute
x = type(x).\__iadd__(x,y): if that method is available and doesn't raise an
exception, that is the result; if it cannot find that method or it raises an
exception, Python executes the code x = x + (y), which also can fail if \__add__
is not defined for the types of x and y.

Here is an example of the \__iadd__ method for the Vector class, which works for
incrementing objects of the Vector type/class by a Vector or an int. Note that
Python automatically will bind x to the result returned by this method.

    def __iadd__(self,right):
        if type(right) not in (Vector,int,float):
            return NotImplemented
        if type(right) in (int,float):
            return Vector(*(c+right for c in self.coords))
        else:
            assert len(self) == len(right), 'Vector.__add__: operand self('+str(self)+') has different dimension that operand right('+str(right)+')'
            return Vector(*(c1+c2 for c1,c2 in zip(self.coords,right.coords)))



In [46]:
x = []
y = x               # x and y share the same empty list
x = x + [1,2]
print(x, y, x is y) # prints [1, 2] [] False

[1, 2] [] False


In [47]:
x = []
y = x               # x and y share the same empty list
x += [1,2]
print(x, y, x is y) # prints [1, 2] [1, 2] True

[1, 2] [1, 2] True


In [52]:
v1 = Vector(0,1)
v2 = Vector(2,2)
v1+=v2

print(v1)
print(v2)

(2)[2, 3]
(2)[2, 2]


## Advanced overloading

So far we've seen some (not all) of the most common operators that can be overloaded. However, we never stopped and asked ourselves why do we want to do overloading? Why not use custom operators instead?. 

This is more of a design question. If you want your custom classes to behave like built in classes (for the sake of usability or to make your code highly compatible) then you'll want your cutom class to include all the operators that people will try to implement on it without raising silly errors.

Now that we have answered this question, we'll move into more advanced overloading. We will not delve too much into the details but instead I will show you all the overloaded methods and then we'll see what they do one by one. However, you should already know that it would be impossible to cover every possible python operator that can be overloaded, therefore if you want more information go into the Python docs.

The contwext of our advanced overloading will be for a cutom list, but this may apply to tuples as well. There are container operators (such as len()) that can be customized for a specific behavior. Let's look at our overload

In [61]:
class List1:
    def __init__(self,_plist):
        self._plist = list(_plist)
        
    def __str__(self):
        return str(self._plist)

    @staticmethod
    def _fix_index(i):
        if i == None:
            return None
        else:
            # for positive indexes, 1 smaller: 1 -> 0
            # for - indexes, the same: -1 (still last), -2 (still 2nd to last
            return (i-1 if i >= 1 else i)
        
    def __getitem__(self,index):
        print('List1.__getitem__('+str(index)+')') # for illumination/debugging
        if type(index) is int:
            return self._plist[List1._fix_index(index)]
        elif type(index) is slice:
            s = slice(List1._fix_index(index.start), List1._fix_index(index.stop), index.step)
            return self._plist[s]
        else:
            raise TypeError('List1.__getitem__ index('+str(index)+') must be int/slice')
  
    def __setitem__(self,index,value):
        print('List1.__setitem__('+str(index)+','+str(value)+')') # for illumination/debugging
        if type(index) is int:
            self._plist[List1._fix_index(index)] = value
        elif type(index) is slice:
            s = slice(List1._fix_index(index.start), List1._fix_index(index.stop), index.step)
            self._plist[s] = value
        else:    
            raise TypeError('List1.__setitem__ index('+str(index)+') must be int/slice')
        
    def __delitem__(self,index):
        print('List1.__delitem__('+str(index)+')') # for illumination/debugging
        if type(index) is int:
            del self._plist[List1._fix_index(index)]
        elif type(index) is slice:
            s = slice(List1._fix_index(index.start), List1._fix_index(index.stop), index.step)
            del self._plist[s]
        else:            
            raise TypeError('List1.__delitem__ index('+str(index)+') must be int/slice')

    def __len__(self):
        return len(self.thislist)
    
    def __contains__(self,item):
        return item in self._plist

First our init and str operators allow us to referrence our list starting with index 1 instead of zero. 

In [62]:
x = List1(['a', 'b', 'c', 'd', 'e'])
print(x)
print(x[1])

['a', 'b', 'c', 'd', 'e']
List1.__getitem__(1)
a


Notice how we need our List1() constructor to specify our custom list. The index starting by 1 is achieve through our static method. 

In Python lists, integer indexes are either non-negative (0, 1, ...),
which specify an index from the beginning (e.g., 0 is the first, 1 is the
second...), or negative, which specify an index from the end (e.g., -1 is the
last index, -2 second from last). Note the asymmetry we are now fixing: in List1
we want 1 to be first and -1 be last, 2 to be second and -2 to be 2nd from last,
unlike Python lists, where 0 is the first index and -1 the last.

So we will start with the helper function (static method) _fix_index: the
leading underscore means this method should be used only other methods in the
List1 class (although it is safe to use by anyone: it doesn't set/mutate any
attributes). This function demotes positive indexes by 1, but leaves 0 and
negative indexes as is. So _fix_index(1) returns 0, which when used to index
self._plist, the delegated list, denotes the index of the first value. Likewise,
_fix_index(-1) returns -1, which still denotes the index of the last value.


In [63]:
print(x[1])
print(x[0])
print(x[-1])

List1.__getitem__(1)
a
List1.__getitem__(0)
a
List1.__getitem__(-1)
e


The next operator is getitem(). This operator uses fix index to address the right item based on the output of our static function. Let'see another example of how this works

In [64]:
x = List1(['a','b','c','d','e'])
print(x)
print(x[1], x[2], x[-2], x[-1])
print(x[1:3])

['a', 'b', 'c', 'd', 'e']
List1.__getitem__(1)
List1.__getitem__(2)
List1.__getitem__(-2)
List1.__getitem__(-1)
a b d e
List1.__getitem__(slice(1, 3, None))
['a', 'b']


Two things about this method. First, we should probably raise an exception if
the index is 0 because that value should not be a legal index in a List1 list.
But for purposes of illustration later, when we discuss the "in" method, we will
not do this (so index 0 will be the same as index 1: both translate to 0).

Also notice how our operator has a specific case for handling slices. If we didn't add this the last print would have raised an error.

We now know how to retrieve values, but how about setting them? We need to overload a different operator called setitem(). It looks quite similar to getitem but with the exception that this writes a value into the list.

In [58]:
x = List1(['a','b','c','d','e'])
print(x)
x[1] = 1
x[4:5] = (4,5)
print(x)

['a', 'b', 'c', 'd', 'e']
List1.__setitem__(1,1)
List1.__setitem__(slice(4, 5, None),(4, 5))
[1, 'b', 'c', 4, 5, 'e']


Next, the \__delitem__(self,index) method is supposed to delete/remove values
from the specified index(es). Its structure is identical to \__getitem__ and
\__set__item, processing int indexes, slice indexes, or raising TypeError. As
with \__setitem__ we automatically return None.

In [59]:
x = List1(['a','b','c','d','e'])
print(x)
del x[1]	# now ['b','c','d','e'] index 1 deleted
print(x)    
del x[2:4]	# now ['b','e']		indexes 2-3 (not 4) deleted
print(x)

['a', 'b', 'c', 'd', 'e']
List1.__delitem__(1)
['b', 'c', 'd', 'e']
List1.__delitem__(slice(2, 4, None))
['b', 'e']


We have already discussed len so we won't go over it again. But contains is interesting. 

We can define our own contains to just use the in operator for lists.

    def __contains__(self,item):
        for v in self._plist:
            if v == item:
                return True
        return False

but this is just checking whether item is in \_plist, so we can simplify it to
delegate to the in operator for standard lists.

    def __contains__(self,item):
        return item in self._plist

In [60]:
x = List1(['a','b','c','d','e'])
print('d' in x)
print('z' in x)

True
False


And that's it. There are other, more complicated methods to be overloaded but they fall under more advanced topics such as attribute methods and inheritance. We will review them when the time comes but for now I don't want to "overload" you with information.